In [1]:
from docx import Document
import numpy as np
from transformers import BertModel, BertTokenizer
import torch 
import json

# Путь к файлу
infile = 'someshit.docx'

# Открываем документ
doc = Document(infile)

# Инициализация токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Сбор текста из документа
all_text = []
structured_tables = []  # Список для таблиц

for paragraph in doc.paragraphs:
    text = paragraph.text.strip()
    if text:
        all_text.append(text)

for table in doc.tables:
    table_data = []
    for row in table.rows:
        row_data = [cell.text.strip() for cell in row.cells]
        table_data.append(row_data)
    structured_tables.append({"data": table_data})

    # Конвертируем строки таблиц в текст
    table_text = [" | ".join(row) for row in table_data]
    all_text.extend(table_text)

# Объединяем текст для токенизации
combined_text = "\n".join(all_text)

# Токенизация текста
inputs = tokenizer(combined_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Получение эмбеддингов из BERT
with torch.no_grad():
    outputs = model(**inputs)

# Вывод скрытых состояний [CLS]-токена
cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()

# Сохранение таблиц без стилей в JSON
with open("tables_structure_no_style.json", "w", encoding="utf-8") as f:
    json.dump(structured_tables, f, ensure_ascii=False, indent=4)

print("Эмбеддинги текста из документа:")
print(combined_text)

/home/mrguss/miniconda3/envs/langchain/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Эмбеддинги текста из документа:
Word Note No. 3.4
Tables
Document release 4.5
Reissued August 2021
Contents
Introduction	1
Macros to create and alter tables	2
Insert an OPC Table macro	2
Add/Remove Rows and Columns macro	2
Inserting or deleting columns	3
Inserting or deleting rows	3
Renumber Table Rows macro	3
Sorting material in tables	3
Format Table Borders macro	4
Align Table Cells macro for aligning numbers in a column	4
Format All Tables macro	5
Note about styles in tables	5
Introduction
This Word Note sets out information about macros to assist staff to produce tables. Rules about tables, and amending tables, in legislation are described in Word Note No. 4.2 (Formatting rules for legislation and other instruments drafted in OPC) and in the Amending Forms Manual.
To assist staff to produce tables, there are macros to:
create new tables;
insert or delete rows in tables;
insert or delete columns in tables;
sort material in tables;
format borders;
number the rows in a table;
format a